In [ ]:
import pypsa
import pandas as pd
from pypsa.descriptors import get_switchable_as_dense
import plotly.graph_objects as go
import numpy as np
import yaml
from matplotlib.colors import to_rgba
pd.set_option('display.max_rows', 150)

In [ ]:
path = "../../pr/"

In [ ]:
with open(path + "pypsa-eur-sec/config.yaml") as file:
    config = yaml.safe_load(file)

colors = config["plotting"]["tech_colors"]

colors["electricity grid"] = "teal"
colors["ground-sourced ambient"] = "orchid"
colors["air-sourced ambient"] = "thistle"

In [ ]:
n = pypsa.Network(
    path + "pypsa-eur-sec/results/your-run-name-overnight-dev/postnetworks/elec_s_60_lv1.25__Co2L0p0-365H-T-H-B-I-solar+p3-dist1_2030.nc"
)

In [ ]:
columns = ["label", "source", "target", "value"]

In [ ]:
gen = (n.snapshot_weightings @ n.generators_t.p).groupby([
    n.generators.carrier, n.generators.carrier, n.generators.bus.map(n.buses.carrier)
]).sum().div(1e6) # TWh

gen.index.set_names(columns[:-1], inplace=True)
gen = gen.reset_index(name='value')
gen = gen.loc[gen.value>0.1]

In [ ]:
gen["source"] = gen["source"].replace({
    "gas": "fossil gas",
    "oil": "fossil oil"
})

In [ ]:
sto = (n.snapshot_weightings @ n.stores_t.p).groupby([
    n.stores.carrier, n.stores.carrier, n.stores.bus.map(n.buses.carrier)
]).sum().div(1e6)
sto.index.set_names(columns[:-1], inplace=True)
sto = sto.reset_index(name='value')
sto = sto.loc[sto.value>.1]

In [ ]:
su = (n.snapshot_weightings @ n.storage_units_t.p).groupby([
    n.storage_units.carrier, n.storage_units.carrier, n.storage_units.bus.map(n.buses.carrier)
]).sum().div(1e6)
su.index.set_names(columns[:-1], inplace=True)
su = su.reset_index(name='value')
su = su.loc[su.value>.1]

In [ ]:
load = (n.snapshot_weightings @ get_switchable_as_dense(n, "Load", "p_set")).groupby([
    n.loads.carrier, n.loads.carrier, n.loads.bus.map(n.buses.carrier)
]).sum().div(1e6).swaplevel() # TWh
load.index.set_names(columns[:-1], inplace=True)
load = load.reset_index(name='value')

In [ ]:
load = load.loc[~load.label.str.contains("emissions")]
load.target += " demand"

In [ ]:
for i in range(5):
    n.links[f"total_e{i}"] = (n.snapshot_weightings @ n.links_t[f"p{i}"]).div(1e6) # TWh
    n.links[f"carrier_bus{i}"] = n.links[f"bus{i}"].map(n.buses.carrier)

In [ ]:
def calculate_losses(x):
    energy_ports = x.loc[
        x.index.str.contains("carrier_bus") &
        ~x.str.contains("co2", na=False)
    ].index.str.replace("carrier_bus", "total_e")
    return -x.loc[energy_ports].sum()

n.links["total_e5"] = n.links.apply(calculate_losses, axis=1)
n.links["carrier_bus5"] = "losses"

In [ ]:
df = pd.concat([
    n.links.groupby(["carrier", "carrier_bus0", "carrier_bus" + str(i)]).sum()["total_e" + str(i)] for i in range(1,6)
]).reset_index()
df.columns = columns

In [ ]:
# fix heat pump energy balance

hp = n.links.loc[n.links.carrier.str.contains("heat pump")]

hp_t_elec = n.links_t.p0.filter(like="heat pump")

hp_elec = (-n.snapshot_weightings @ hp_t_elec).groupby([hp["carrier"], hp["carrier_bus0"], hp["carrier_bus1"]]).sum().div(1e6).reset_index()
hp_elec.columns = columns

df = df.loc[~(df.label.str.contains("heat pump") & (df.target == 'losses'))]

df.loc[df.label.str.contains("heat pump"), "value"] -= hp_elec["value"].values

df.loc[df.label.str.contains("air heat pump"), "source"] = "air-sourced ambient"
df.loc[df.label.str.contains("ground heat pump"), "source"] = "ground-sourced ambient"

df = pd.concat([df, hp_elec])

In [ ]:
df = df.set_index(["label", "source", "target"]).squeeze()

In [ ]:
df = pd.concat([
    df.loc[df<0].mul(-1),
    df.loc[df>0].swaplevel(1, 2),
]).reset_index()

df.columns = columns

In [ ]:
# make DAC demand
df.loc[df.label=='DAC', "target"] = "DAC"

In [ ]:
connections = pd.concat([
    df,
    gen,
    su,
    sto,
    load,
]).sort_index().reset_index(drop=True)

In [ ]:
# aggregation

src_contains = connections.source.str.contains
trg_contains = connections.target.str.contains

connections.loc[src_contains("low voltage"), "source"] = "AC"
connections.loc[trg_contains("low voltage"), "target"] = "AC"
connections.loc[src_contains("water tank"), "source"] = "water tank"
connections.loc[trg_contains("water tank"), "target"] = "water tank"
connections.loc[src_contains("solar thermal"), "source"] = "solar thermal"
connections.loc[src_contains("battery"), "source"] = "battery"
connections.loc[trg_contains("battery"), "target"] = "battery"
connections.loc[src_contains("Li ion"), "source"] = "battery"
connections.loc[trg_contains("Li ion"), "target"] = "battery"

connections.loc[src_contains("heat") & ~src_contains("demand"), "source"] = "heat"
connections.loc[trg_contains("heat") & ~trg_contains("demand"), "target"] = "heat"

In [ ]:
connections = connections.loc[
    ~(connections.source == connections.target) 
    & ~connections.source.str.contains("co2")
    & ~connections.target.str.contains("co2")
    & ~connections.source.str.contains("emissions")
    & ~connections.source.isin(['gas for industry', "solid biomass for industry"])
    & (connections.value >= 0.5)
]

In [ ]:
where = connections.label=='urban central gas boiler'
connections.loc[where] = connections.loc[where].replace("losses", "fossil gas")

In [ ]:
connections.replace("AC", "electricity grid", inplace=True)

In [ ]:
labels = np.unique(connections[["source", "target"]])

In [ ]:
nodes = pd.Series({v: i for i, v in enumerate(labels)})

In [ ]:
node_colors = pd.Series(nodes.index.map(colors).fillna("grey"), index=nodes.index)

In [ ]:
link_colors = ["rgba{}".format(to_rgba(node_colors[src], alpha=0.5)) for src in connections.source]

In [ ]:
fig = go.Figure(go.Sankey(
    arrangement="snap", # [snap, nodepad, perpendicular, fixed]
    valuesuffix = "TWh",
    valueformat = ".1f",
    node = dict(
        pad=15,
        thickness=10,
        label=nodes.index,
        color=node_colors
    ),
    link = dict(
        source=connections.source.map(nodes),
        target=connections.target.map(nodes),
        value=connections.value,
        label=connections.label,
        color=link_colors,
    )
))

fig.update_layout(
    title="Sankey Diagram: PyPSA-Eur-Sec",
    font_size=15
)

fig.write_html("Co2L0p0.html")